In [308]:
import pandas as pd
import json

In [310]:
df = pd.read_json('case.json')

In [312]:
df.sample(10)

,EnqueuedTimeUtc,EventName,Payload
28,2021-09-05 09:02:21 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
9,2021-09-05 09:18:04 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
4,2021-09-05 08:03:28 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
22,2021-08-18 11:44:12 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
25,2021-09-05 09:02:05 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
5,2021-08-18 11:44:00 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
12,2021-08-18 11:44:04 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
10,2021-09-05 08:04:16 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
32,2021-08-18 11:44:08 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
27,2021-09-05 09:02:17 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."


In [314]:
df.shape

(37, 3)

In [316]:
df['EnqueuedTimeUtc'] = pd.to_datetime(df['EnqueuedTimeUtc'])

In [318]:
df['EnqueuedTimeUtc'] = df['EnqueuedTimeUtc'].dt.tz_convert(tz='Brazil/East')

In [320]:
df.dtypes

EnqueuedTimeUtc    datetime64[ns, Brazil/East]
EventName                               object
Payload                                 object
dtype: object

# DynamicPriceOption.csv
Provider: in quotes

OfferId: in quotes

UniqueOptionId: in quotes

BestPrice: without quotes

EnqueuedTimeSP: DD/MM/YYYY (converted to Brasilian timezone - UTC-3)

"{

"provider":"ApplyDynamicPricePerOption",

"offerId":"56e0702c-0218-4626-8d3d-ae9d54b4503b",

"algorithmOutput":[
{"uniqueOptionId":"b0e296a9-0590-f0e0-8211-243a2ededb12","bestPrice":92.45},
{"uniqueOptionId":"d6562c24-0b37-5fb4-8275-65b7b8b47b87","bestPrice":92.45},
{"uniqueOptionId":"8d0f9262-f543-d0c8-a869-33985ae3ecda","bestPrice":92.45},
{"uniqueOptionId":"151e59ac-761a-96f5-d2b9-882037a9fd28","bestPrice":94.6},
{"uniqueOptionId":"3cd346f4-d297-7568-2e50-d43a8e2fd0a9","bestPrice":94.6},
{"uniqueOptionId":"b7a7b6d1-4dae-7392-5aaf-f3369c29db1d","bestPrice":93.0},
{"uniqueOptionId":"577e4bbd-f49d-ac23-56a6-e70072a05229","bestPrice":93.0},
{"uniqueOptionId":"f9b876ab-2590-952f-d69d-5b352ec251f3","bestPrice":91.35}]


}"

In [324]:
DynamicPriceEvents = df [df['EventName'] == 'DynamicPrice_Result']
DynamicPriceEvents.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 05:04:08-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 08:43:23-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 06:04:04-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
4,2021-09-05 05:03:28-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
5,2021-08-18 08:44:00-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."


In [326]:
DynamicPriceOption.shape

(32, 3)

In [370]:
def dynamic_price_option(row):
    payload_row = row['Payload']
    payload_row = json.loads(payload_row)
    enqueuedTime = row['EnqueuedTimeUtc'].to_pydatetime().strftime('%d/%m/%Y')
    result = [] 
    if payload_row['provider'] == 'ApplyDynamicPricePerOption':
        provider = f"\"{payload_row['provider']}\""
        offerID = f"\"{payload_row['offerId']}\""
        for unique_option in payload_row['algorithmOutput']:
            price_option = {
                'Provider': provider,
                'OfferId': offerID,
                'UniqueOptionId': f"\"{unique_option['uniqueOptionId']}\"",
                'BestPrice': f"{unique_option['bestPrice']}",
                'EnqueuedTimeSP': enqueuedTime
            }
            result.append(price_option)
    return result

In [372]:
lst = DynamicPriceEvents.apply(dynamic_price_option, axis = 1)

In [374]:
flatten = []
for sublist in lst:
    for x in sublist:
        flatten.append(x)
df_dynamic_price_option = pd.DataFrame(flatten)

In [376]:
df_dynamic_price_option.sample(10)

,Provider,OfferId,UniqueOptionId,BestPrice,EnqueuedTimeSP
22,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""577e4bbd-f49d-ac23-56a6-e70072a05229""",93.0,18/08/2021
17,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,18/08/2021
40,"""ApplyDynamicPricePerOption""","""6dc306e7-7a6a-459a-a12c-400b4b3c9d37""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,18/08/2021
19,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""151e59ac-761a-96f5-d2b9-882037a9fd28""",94.6,18/08/2021
32,"""ApplyDynamicPricePerOption""","""7cb20a39-c7f9-4c4c-9d51-94dbb8c4ff07""","""9c4ec643-934f-56d1-198d-46a8adc6c405""",93.5,18/08/2021
3,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""151e59ac-761a-96f5-d2b9-882037a9fd28""",94.6,18/08/2021
23,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""f9b876ab-2590-952f-d69d-5b352ec251f3""",91.35,18/08/2021
38,"""ApplyDynamicPricePerOption""","""7cb20a39-c7f9-4c4c-9d51-94dbb8c4ff07""","""577e4bbd-f49d-ac23-56a6-e70072a05229""",94.4,18/08/2021
5,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b7a7b6d1-4dae-7392-5aaf-f3369c29db1d""",93.0,18/08/2021
12,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""",94.6,18/08/2021


In [392]:
df_dynamic_price_option.to_csv('DynamicPriceOption.csv', index = False)

In [336]:
df_dynamic_price_option.shape

(56, 5)

# DynamicPriceRange.csv
Provider: "in quotes"

OfferId: "in quotes"

MinGlobal: without quotes

MinRecommended: without quotes

MaxRecommended: without quotes

DifferenceMinRecommendMinTheory: without quotes

EnqueuedTimeSP: DD/MM/YYYY (converted to Brazilian timezone - UTC-3)

## sample_row:

EnqueuedTimeUtc"2021-09-05 09:04:04 UTC"

EventName"DynamicPrice_Result"



Payload"{
"provider":"ApplyDynamicPriceRange",

"offerId":"b8c636fa-8241-47dc-ac40-bdf438a04d9c",

"algorithmOutput":{"min_global":85.0,

"min_recommended":87.2,

"max_recommended":97.65,

"differenceMinRecommendMinTheory":2.2}
}""}"

In [378]:
def dynamic_price_range (row):
    enqueuedTime = row['EnqueuedTimeUtc'].to_pydatetime().strftime('%d/%m/%Y')
    payload_row = row['Payload']
    payload_row = json.loads(payload_row)
    result = []
    if payload_row['provider'] == 'ApplyDynamicPriceRange':
        price_range = {
            'Provider': f'"{payload_row['provider']}"',
            'OfferId': f'"{payload_row['offerId']}"', 
            'MinGlobal': f'{payload_row['algorithmOutput']['min_global']}',   
            'MinRecommended': f'{payload_row['algorithmOutput']['min_recommended']}',   
            'MaxRecommended': f'{payload_row['algorithmOutput']['max_recommended']}',   
            'DifferenceMinRecommendMinTheory': f'{payload_row['algorithmOutput']['differenceMinRecommendMinTheory']}',
            'EnqueuedTimeSP': enqueuedTime
        }
        result.append(price_range)
    return result

In [380]:
lst = DynamicPriceEvents.apply(dynamic_price_range, axis = 1)

In [382]:
lst.sample(5)

19    [{'Provider': '"ApplyDynamicPriceRange"', 'Off...
7     [{'Provider': '"ApplyDynamicPriceRange"', 'Off...
5                                                    []
23    [{'Provider': '"ApplyDynamicPriceRange"', 'Off...
0     [{'Provider': '"ApplyDynamicPriceRange"', 'Off...
dtype: object

In [384]:
flatten_range = []
for sublist in lst:
    for x in sublist:
        flatten_range.append(x)
        
df_DynamicPriceRange = pd.DataFrame(flatten_range)


In [386]:
df_DynamicPriceRange.sample(10)

,Provider,OfferId,MinGlobal,MinRecommended,MaxRecommended,DifferenceMinRecommendMinTheory,EnqueuedTimeSP
5,"""ApplyDynamicPriceRange""","""27bbc4fa-2388-4780-b66c-92a51397d191""",87.35,89.25,99.95,1.9,05/09/2021
10,"""ApplyDynamicPriceRange""","""e94cf567-d77a-4874-9663-0b49d19f604c""",85.0,87.2,97.65,2.2,05/09/2021
0,"""ApplyDynamicPriceRange""","""a6611d55-9624-4381-8cdd-323ee3689241""",85.0,87.2,97.65,2.2,05/09/2021
24,"""ApplyDynamicPriceRange""","""92269c83-9103-4ea3-b93b-ae2f1762f720""",85.0,87.2,97.65,2.2,05/09/2021
3,"""ApplyDynamicPriceRange""","""329194f3-95a4-45ef-b3d0-2796f74ce2a0""",85.0,87.2,97.65,2.2,05/09/2021
15,"""ApplyDynamicPriceRange""","""4e44bf19-62ff-4d6e-8ce6-f4de80bb027d""",85.0,87.2,97.65,2.2,05/09/2021
14,"""ApplyDynamicPriceRange""","""d7739a03-c1f8-46db-971f-1a9af3dd88d9""",87.35,89.25,99.95,1.9,05/09/2021
2,"""ApplyDynamicPriceRange""","""3d32f7fb-396d-4d3f-b673-dea1f7dc41b7""",85.0,87.2,97.65,2.2,05/09/2021
22,"""ApplyDynamicPriceRange""","""c0b7cf25-62d1-4f3d-be18-3825c316ed8d""",85.0,87.2,97.65,2.2,05/09/2021
6,"""ApplyDynamicPriceRange""","""baffc30b-7642-45fe-a2ce-da31a71732ae""",85.0,87.2,97.65,2.2,05/09/2021


In [388]:
df_DynamicPriceRange.to_csv('DynamicPriceRange.csv', index=False)

# CuratedOfferOptions.csv:

CurationProvider: in quotes

OfferId: in quotes

DealerId: in quotes

UniqueOptionId: in quotes

OptionId: in quotes

IsMobileDealer: without quotes

IsOpen: without quotes

Eta: in quotes

ChamaScore: without quotes

ProductBrand: in quotes

IsWinner: without quotes

MinimumPrice: without quotes

MaximumPrice: without quotes

DynamicPrice: without quotes

FinalPrice: without quotes

DefeatPrimaryReason: in quotes

DefeatReasons: in quotes

EnqueuedTimeSP: DD/MM/YYYY (converted to Brasilian timezone - UTC-3)


EnqueuedTimeUtc"2021-08-25 05:02:55 UTC"

EventName"CurateOffer_Result
"
Payload"[{"curationProvider":"ByPrice
        ","offerId":"149f0e53-ff85-425f-a01a-8710f06704ea
        ","dealerId":"6517
        ","options":[{"uniqueOptionId":"b0e296a9-0590-f0e0-8211-243a2ededb12
                    ","optionId":"6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2
                    ","isMobileDealer":tru
                    e,"isOpen":tru
                    e,"eta":"1:00
                    ","chamaScore":8.0,"productBrand":"ULTRAGAZ
                    ","isWinner":tru
                    e,"minimumPrice":90.0
                    0,"maximumPrice":180.0
                    0,"dynamicPrice":91.9
                    0,"finalPrice":91.90
                    },{"uniqueOptionId":"d6562c24-0b37-5fb4-8275-65b7b8b47b87
                    ","optionId":"6517 || 6517
                    ","isMobileDealer":fals
                    e,"isOpen":false,"eta":"0:01
                    ","chamaScore":8.
                    0,"productBrand":"ULTRAGAZ
                    ","isWinner":fals
                    e,"minimumPrice":90.0
                    0,"maximumPrice":180.0
                    0,"dynamicPrice":91.9
                    0,"finalPrice":91.9
                    0,"defeatPrimaryReason":"Closed
                    ","defeatReasons":["Closed","HasDriverInOffer"]}]
]"]"

In [470]:
def curateoffer_option(row):
    enqueuedTime = row['EnqueuedTimeUtc'].to_pydatetime().strftime('%d/%m/%Y')
    payload_row = row['Payload']
    payload_row = json.loads(payload_row)
    result = []
    for curation_offer in payload_row:
        CurationProvider = f'"{curation_offer['curationProvider']}"'
        OfferId = f'"{curation_offer['offerId']}"'
        DealerId = f'"{curation_offer['dealerId']}"'
        for unique_options in curation_offer['options']:
            offer_option = {
                'CurationProvider' : CurationProvider,
                'OfferId' : OfferId,
                'DealerId' : DealerId,
                'UniqueOptionId' : f'"{unique_options['uniqueOptionId']}"',
                'OptionId' : f'"{unique_options['optionId']}"',
                'IsMobileDealer' : f'{unique_options['isMobileDealer']}',
                'IsOpen' : f'{unique_options['isOpen']}',
                'Eta' : f'"{unique_options['eta']}"',
                'ChamaScore' : f'{unique_options['chamaScore']}',
                'ProductBrand' : f'"{unique_options['productBrand']}"',
                'IsWinner' : f'{unique_options['isWinner']}',
                'MinimumPrice' : f'{unique_options['minimumPrice']}',
                'MaximumPrice' : f'{unique_options['maximumPrice']}',
                'DynamicPrice' : f'{unique_options['dynamicPrice']}',
                'FinalPrice' : f'{unique_options['finalPrice']}',
            }
            if 'defeatPrimaryReason' in unique_options:
                offer_option['DefeatPrimaryReason'] = f'"{unique_options['defeatPrimaryReason']}"'
                offer_option['DefeatReasons'] = f'"{unique_options['defeatReasons']}"'
            else:
                offer_option['DefeatPrimaryReason'] = f'""'
                offer_option['DefeatReasons'] = f'""'
            offer_option['EnqueuedTimeSP'] =  enqueuedTime
            result.append(offer_option)
    return result

In [472]:
CuratedOffer_Result = df [df['EventName'] == 'CurateOffer_Result']

In [474]:
CuratedOffer_Result

,EnqueuedTimeUtc,EventName,Payload
3,2021-08-25 02:02:55-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
13,2021-08-25 06:03:29-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""c99a..."
15,2021-08-25 02:02:18-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""d668..."
17,2021-08-25 06:02:29-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""0a06..."
35,2021-08-25 06:03:14-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""135d..."


In [476]:
lst =  CuratedOffer_Result.apply(curateoffer_option, axis = 1)

In [478]:
flatten_curatedoffer = []
for sublist in lst:
    for x in sublist:
        flatten_curatedoffer.append(x)
CuratedOfferOptions = pd.DataFrame(flatten_curatedoffer)
CuratedOfferOptions

,CurationProvider,OfferId,DealerId,UniqueOptionId,OptionId,IsMobileDealer,IsOpen,Eta,ChamaScore,ProductBrand,IsWinner,MinimumPrice,MaximumPrice,DynamicPrice,FinalPrice,DefeatPrimaryReason,DefeatReasons,EnqueuedTimeSP
0,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""6517""","""b0e296a9-0590-f0e0-8211-243a2ededb12""","""6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2""",True,True,"""1:00""",8.0,"""ULTRAGAZ""",True,90.0,180.0,91.9,91.9,"""""","""""",25/08/2021
1,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""6517""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""","""6517 || 6517""",False,False,"""0:01""",8.0,"""ULTRAGAZ""",False,90.0,180.0,91.9,91.9,"""Closed""","""['Closed', 'HasDriverInOffer']""",25/08/2021
2,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""","""9047 || 9047 || ULTRAGAZ""",False,False,"""1:00""",9.0,"""ULTRAGAZ""",False,99.0,198.0,99.95,99.95,"""Closed""","""['Closed', 'HigherPrice', 'HasDriverInOffer']""",25/08/2021
3,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""","""9047 || 9047 || CONSIGAZ""",False,False,"""1:00""",9.0,"""CONSIGAZ""",False,89.99,179.98,91.89,91.89,"""Closed""","""['Closed', 'HigherPrice', 'HigherETA']""",25/08/2021
4,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""577e4bbd-f49d-ac23-56a6-e70072a05229""","""9047 || 9047 || LIQUIGAS""",False,False,"""1:00""",9.0,"""LIQUIGAS""",False,92.0,184.0,93.9,93.9,"""Closed""","""['Closed', 'HigherPrice', 'HigherETA']""",25/08/2021
5,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""f9b876ab-2590-952f-d69d-5b352ec251f3""","""9047 || 9047 || NACIONALGAS""",False,False,"""1:00""",9.0,"""NACIONALGAS""",True,88.9,177.8,90.8,90.8,"""""","""""",25/08/2021
6,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""8330""","""151e59ac-761a-96f5-d2b9-882037a9fd28""","""8330 || 8330 || CONSIGAZ""",False,True,"""0:01""",8.0,"""CONSIGAZ""",True,80.0,160.0,87.35,87.35,"""""","""""",25/08/2021
7,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""8835""","""b7a7b6d1-4dae-7392-5aaf-f3369c29db1d""","""8835 || 8835 || LIQUIGAS""",False,False,"""0:01""",8.0,"""LIQUIGAS""",True,60.0,120.0,87.35,87.35,"""""","""""",25/08/2021
8,"""ByPrice""","""c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d""","""6517""","""b0e296a9-0590-f0e0-8211-243a2ededb12""","""6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2""",True,True,"""00:12:54.9215999""",8.0,"""ULTRAGAZ""",True,90.0,180.0,91.9,91.9,"""""","""""",25/08/2021
9,"""ByPrice""","""c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d""","""6517""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""","""6517 || 6517""",False,True,"""0:01""",8.0,"""ULTRAGAZ""",False,90.0,180.0,91.9,91.9,"""HasDriverInOffer""","""['HasDriverInOffer']""",25/08/2021


In [480]:
CuratedOfferOptions.to_csv('curated_offers_payload.csv', index = False)